# Training_data_task_Data_prep

 * `TrainingDataTask`: Extracts features/outcome variable in preparation for training a model.
    * This prepares the cleaned data into the exact form that is able to be fit by the model.
    * The "y" variable will be the multi-class sentiment (0, 1, 2 for negative, neutral and positive respectively).
    * The "X" variables will be the closest city to the "tweet_coord" using Euclidean distance.
    * You should use the `cities.csv` file to find the closest city.
    * You probably will need to one-hot encode the city names.



In [2]:
import sys
import pandas as pd
from modules.helpers import read_in_dataset
import inspect
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer

print(inspect.getsource(read_in_dataset))

/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


def read_in_dataset(dset, verbose=False):
    
    """Read in one of the datasets (train or properties)
        
        Keyword arguments:
        dset -- a string
        verbose -- whether or not to print info about the dataset
        
        Returns:
        a pandas dataframe
        """
    
    df =  pd.read_csv('{0}.csv'.format(dset), encoding = "ISO-8859-1")
    
    if verbose:
        print('\n{0:*^80}'.format(' Reading in the {0} dataset '.format(dset)))
        print("\nit has {0} rows and {1} columns".format(*df.shape))
        print('\n{0:*^80}\n'.format(' It has the following columns '))
        print(df.columns)
        print('\n{0:*^80}\n'.format(' The first 5 rows look like this '))
        print(df.head())
    
    return df



/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
cleaned_data_df = read_in_dataset('clean_data', verbose = True)
cities_df = read_in_dataset('cities', verbose= True)


********************** Reading in the clean_data dataset ***********************

it has 855 rows and 20 columns

************************* It has the following columns *************************

Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'airline_sentiment',
       'airline_sentiment:confidence', 'negativereason',
       'negativereason:confidence', 'airline', 'airline_sentiment_gold',
       'name', 'negativereason_gold', 'retweet_count', 'text', 'tweet_coord',
       'tweet_created', 'tweet_id', 'tweet_location', 'user_timezone'],
      dtype='object')

*********************** The first 5 rows look like this ************************

    _unit_id  _golden _unit_state  _trusted_judgments _last_judgment_at  \
0  681448197    False   finalized                   3      2/25/15 2:26   
1  681448213    False   finalized                   3      2/25/15 9:04   
2  681448214    False   finalized                   3      2/25/15 9:14   
3 

# Prepare the cities_df  in the right format for training

- The "X" variables will be the closest city to the "tweet_coord" using Euclidean distance.
 - use the `cities.csv` file to find the closest city.

In [4]:
cities_df.head()

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date
0,3040051,les Escaldes,les Escaldes,"Ehskal'des-Ehndzhordani,Escaldes,Escaldes-Engo...",42.50729,1.53414,P,PPLA,AD,NaN,08,NaN,NaN,NaN,15853,NaN,1033,Europe/Andorra,2008-10-15
1,3041563,Andorra la Vella,Andorra la Vella,"ALV,Ando-la-Vyey,Andora,Andora la Vela,Andora ...",42.50779,1.52109,P,PPLC,AD,NaN,07,NaN,NaN,NaN,20430,NaN,1037,Europe/Andorra,2010-05-30
2,290594,Umm al Qaywayn,Umm al Qaywayn,"Oumm al Qaiwain,Oumm al QaÃ¯waÃ¯n,Um al Kawain...",25.56473,55.55517,P,PPLA,AE,NaN,07,NaN,NaN,NaN,44411,NaN,2,Asia/Dubai,2014-10-07
3,291074,Ras al-Khaimah,Ras al-Khaimah,"Julfa,Khaimah,RKT,Ra's al Khaymah,Ra's al-Chai...",25.78953,55.94320,P,PPLA,AE,NaN,05,NaN,NaN,NaN,115949,NaN,2,Asia/Dubai,2015-12-05
4,291696,Khawr FakkÄn,Khawr Fakkan,"Fakkan,FakkÄn,Khawr Fakkan,Khawr FakkÄn,Khaw...",25.33132,56.34199,P,PPL,AE,NaN,06,NaN,NaN,NaN,33575,NaN,20,Asia/Dubai,2013-10-25


### Explore the Null values

In [5]:
cities_df.latitude.isna().sum()

0

In [6]:
cities_df.longitude.isna().sum()

0

In [7]:
cities_df.isnull().sum().sort_values(ascending=True)  # no missing values in the relevant columns
# no missing values in the relevant columns


geonameid                0
dem                      0
population               0
timezone                 0
feature code             0
feature class            0
modification date        0
latitude                 0
asciiname                0
name                     0
longitude                0
admin1 code              7
country code            13
alternatenames        1997
admin2 code           7211
admin3 code          16252
elevation            19592
admin4 code          21065
cc2                  23036
dtype: int64

In [8]:
cities_df.latitude.dtypes

dtype('float64')

In [11]:
cleaned_data_df.tweet_coord.head()

0     [40.74804263, -73.99295302]
1       [42.361016, -71.02000488]
2     [33.94540417, -118.4062472]
3    [33.94209449, -118.40410103]
4       [33.2145038, -96.9321504]
Name: tweet_coord, dtype: object

###  Find the the closest city to the "tweet_coord" using Euclidean distance

- and map them with the cleaned_data_df, merge the name column from the cities dataframe with cleaned_data_df
    - find the distance for each of cleaned_data_df with cities_coordinates and sort them based on the euclidian distance

In [12]:
 from ast import literal_eval


### first the find the closest city for the first tweet coordinate

In [13]:
import numpy as np

def dist(lat1, long1, lat2, long2):
    """
     the function that calculates the euclidian distance of the lat and long of two points
    """
    return np.sqrt((lat1-lat2)**2+(long1-long2)**2)
 

In [15]:
# first the find the closest city for the first tweet coordinate
def find_closest_city(lat, long):
    
    distances = cities_df.apply(lambda row: dist(lat, long, row['latitude'], row['longitude']), axis=1)
    
    return cities_df.loc[distances.idxmin(), 'name']

In [14]:
lat = 40.74804263
long = -73.99295302

In [16]:
find_closest_city(lat, long)

'New York City'

- find the corresponding closed city for each tweet in the cleaned_data_df
- the function literal_eval removes the string quotes and coonvert the [lat, long] in float type

In [17]:
from ast import literal_eval
closest_cities = cleaned_data_df.apply(
    lambda row: find_closest_city(literal_eval(row['tweet_coord'])[0], literal_eval(row['tweet_coord'])[1]), 
    axis=1)

In [18]:
closest_cities

0            New York City
1                  Chelsea
2               El Segundo
3               El Segundo
4                   Frisco
5              Culver City
6              Aliso Viejo
7      Springfield Gardens
8                 Paradise
9            San Francisco
10             Dania Beach
11              El Segundo
12                 Calgary
13     Springfield Gardens
14              El Segundo
15        Washington, D.C.
16                  Dallas
17                  Dallas
18             Dania Beach
19                  Dallas
20               Gladstone
21           San Francisco
22              El Segundo
23           Port Richmond
24              Ronkonkoma
25                Westford
26           San Francisco
27         Summerlin South
28     Venustiano Carranza
29                Brighton
              ...         
825              Charlotte
826           Santa Monica
827    Springfield Gardens
828    Springfield Gardens
829              Charlotte
830                Coppell
8

In [19]:
type(closest_cities)

pandas.core.series.Series

### Create a new column at the cleaned _data_df with the corresponding closest cities

In [45]:
cleaned_data_df['closest_cities'] = closest_cities
cleaned_data_df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,airline_sentiment,airline_sentiment:confidence,negativereason,negativereason:confidence,airline,...,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone,closest_cities,sentiment
0,681448197,False,finalized,3,2/25/15 2:26,positive,1.000,NaN,NaN,Virgin America,...,NaN,0,@VirginAmerica I love this graphic. http://t.c...,"[40.74804263, -73.99295302]",2/24/15 8:49,5.702640e+17,NaN,NaN,New York City,2.0
1,681448213,False,finalized,3,2/25/15 9:04,negative,1.000,Bad Flight,1.0,Virgin America,...,NaN,0,@VirginAmerica amazing to me that we can't get...,"[42.361016, -71.02000488]",2/24/15 5:05,5.702080e+17,"San Mateo, CA & Las Vegas, NV",NaN,Chelsea,0.0
2,681448214,False,finalized,3,2/25/15 9:14,neutral,0.615,NaN,0.0,Virgin America,...,NaN,0,@VirginAmerica LAX to EWR - Middle seat on a r...,"[33.94540417, -118.4062472]",2/23/15 23:34,5.701250e+17,Brooklyn,Atlantic Time (Canada),El Segundo,1.0
3,681448223,False,finalized,3,2/25/15 1:57,negative,1.000,Customer Service Issue,1.0,Virgin America,...,NaN,0,"@VirginAmerica help, left expensive headphones...","[33.94209449, -118.40410103]",2/23/15 21:10,5.700880e+17,Washington DC,Quito,El Segundo,0.0
4,681448228,False,finalized,3,2/25/15 1:01,positive,1.000,NaN,NaN,Virgin America,...,NaN,0,@VirginAmerica this is great news! America co...,"[33.2145038, -96.9321504]",2/23/15 20:24,5.700770e+17,Texas,Central Time (US & Canada),Frisco,2.0


# Create the Y

In [46]:
# create a new column that maps (0, 1, 2 for negative, neutral and positive respectively) of ariline_sentiment
cleaned_data_df.loc[cleaned_data_df['airline_sentiment']=='negative', 
             'sentiment'] = 0
cleaned_data_df.loc[cleaned_data_df['airline_sentiment']=='neutral', 
             'sentiment'] = 1
cleaned_data_df.loc[cleaned_data_df['airline_sentiment']=='positive', 
             'sentiment'] = 2


In [47]:
Y = cleaned_data_df.sentiment
Y.head()

0    2.0
1    0.0
2    1.0
3    0.0
4    2.0
Name: sentiment, dtype: float64

# Write the prepared data for training to file

In [48]:
cleaned_data_df.to_csv('training_data.csv', index=False)

In [49]:
from sklearn.model_selection import train_test_split
X = cleaned_data_df.closest_cities
y = cleaned_data_df.sentiment
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
encoder = LabelBinarizer()
X_lb = encoder.fit_transform(X)
X_lb.shape

(855, 291)

In [51]:
type(y)

pandas.core.series.Series

# Write to file
Output file should have columns corresponding to the training data:
        - y = airline_sentiment (coded as 0=negative, 1=neutral, 2=positive)
        - X = a one-hot coded column for each city in "cities.csv"

In [52]:
#create a dataframe: that contains X_lb and Y
x = X_lb.tolist()
y = y.to_list()

features_df = pd.DataFrame(
    {'X': x,
     'y': y,
    })


In [53]:
features_df.to_csv('features.csv', index=False)